In [ ]:
import cv2
import numpy as np

In [ ]:
# Read the source and guide images
source_image = cv2.imread('2_a.jpg')
guide_image = cv2.imread('2_b.jpg')

In [ ]:
# Set the parameters for the bilateral filter
diameter = 9  # Diameter of each pixel neighborhood
sigma_color = 0.5  # Filter sigma in the color space
sigma_space = 40  # Filter sigma in the coordinate space

In [ ]:
def bilateral_filter(input_image, guide_image, diameter, sigma_color, sigma_space):
    output_image = np.zeros_like(input_image, dtype=np.float32)

    height, width, channels = input_image.shape

    for i in range(height):
        for j in range(width):
            pixel_value = input_image[i, j]

            # Define the window for filtering
            i_min = max(0, i - diameter // 2)
            i_max = min(height, i + diameter // 2 + 1)
            j_min = max(0, j - diameter // 2)
            j_max = min(width, j + diameter // 2 + 1)

            # Apply the filter within the window
            for x in range(i_min, i_max):
                for y in range(j_min, j_max):
                    spatial_weight = np.exp(-((i - x)2 + (j - y)*2) / (2 * sigma_space*2))
                    color_weight = np.exp(-np.linalg.norm(guide_image[i, j] - guide_image[x, y])*2 / (2 * sigma_color*2))
                    weight = spatial_weight * color_weight
                    output_image[i, j] += weight * input_image[x, y]

            # Normalize the filtered value
            output_image[i, j] /= np.sum(weight)

    return output_image.astype(np.uint8)

In [ ]:
# Convert images to float32
source_image_float32 = source_image.astype(np.float32) / 255.0
guide_image_float32 = guide_image.astype(np.float32) / 255.0

In [ ]:
# Apply the bilateral filter
filtered_image = bilateral_filter(source_image_float32, guide_image_float32, diameter, sigma_color, sigma_space)

In [ ]:
# Denoise the filtered image
denoised_image = cv2.fastNlMeansDenoisingColored(filtered_image, None, 10, 10, 7, 21)

In [ ]:
# Display the source, guide, filtered, and denoised images
cv2.imshow('Source Image', source_image)
cv2.imshow('Guide Image', guide_image)
cv2.imshow('Filtered Image', filtered_image)
cv2.imshow('Denoised Image', denoised_image)
cv2.imwrite('dullf.jpg', denoised_image)
cv2.waitKey(0)
cv2.destroyAllWindows()